# Computer Vision: Calibrating Image Classification Models

This example demonstrates rank-preserving calibration for computer vision applications using the handwritten digits dataset. We'll show how classifiers often suffer from overconfidence and how calibration can improve reliability while maintaining predictive performance.

## Computer Vision Motivation

Deep learning models for image classification face several calibration challenges:
- **Overconfidence**: Neural networks often produce overly confident predictions
- **Dataset shift**: Models trained on one dataset may be poorly calibrated on another
- **Class imbalance**: Real-world deployments often have different class distributions than training data
- **Safety-critical applications**: Medical imaging, autonomous vehicles require well-calibrated uncertainty

Rank-preserving calibration maintains the model's ability to distinguish between images while providing more reliable probability estimates.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import calibration_curve
from scipy.stats import entropy
import warnings
warnings.filterwarnings('ignore')

# Import our calibration package - proper imports
from rank_preserving_calibration import calibrate_dykstra

# Set style for publication-quality plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 11

## Dataset and Business Context

We'll use the handwritten digits dataset to simulate an optical character recognition (OCR) system deployed across different contexts:

1. **Training environment**: Controlled lab conditions with balanced digit distribution
2. **Production environment**: Real-world ZIP code processing with skewed digit frequencies

The calibration challenge: ZIP codes contain certain digits more frequently (like 0, 1, 2) than others (like 8, 9).

In [ ]:
# Load the handwritten digits dataset
print("📊 LOADING HANDWRITTEN DIGITS DATASET")
print("="*60)

digits = load_digits()
X, y = digits.data, digits.target

print(f"Dataset shape: {X.shape}")
print(f"Number of classes: {len(np.unique(y))}")
print(f"Feature dimensions: {X.shape[1]} (8x8 grayscale images)")

# Show class distribution in training data
training_distribution = np.bincount(y) / len(y)
print(f"\nTraining class distribution (balanced):")
for digit, freq in enumerate(training_distribution):
    print(f"  Digit {digit}: {freq:.3f} ({freq*100:.1f}%)")

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"\nTraining samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

## Model Training and Initial Evaluation

In [ ]:
# Train a Random Forest classifier
print("🤖 TRAINING COMPUTER VISION MODEL")
print("="*50)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
model = RandomForestClassifier(
    n_estimators=100, 
    max_depth=10, 
    random_state=42,
    class_weight='balanced'  # Help with any residual imbalance
)
model.fit(X_train_scaled, y_train)

# Get predictions and probabilities
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)

# Baseline performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Model accuracy: {accuracy:.3f}")
print(f"Number of test samples: {len(y_test)}")

# Check prediction confidence
max_probas = np.max(y_proba, axis=1)
print(f"\nPrediction confidence statistics:")
print(f"  Mean max probability: {np.mean(max_probas):.3f}")
print(f"  Median max probability: {np.median(max_probas):.3f}")
print(f"  Std max probability: {np.std(max_probas):.3f}")
print(f"  Min max probability: {np.min(max_probas):.3f}")
print(f"  Max max probability: {np.max(max_probas):.3f}")

# Show current marginals (class frequencies in predictions)
current_marginals = np.mean(y_proba, axis=0)
print(f"\nCurrent probability marginals:")
for digit, marginal in enumerate(current_marginals):
    print(f"  Digit {digit}: {marginal:.3f}")

## Production Deployment Scenario

Now we simulate deploying this model to process ZIP codes, where digit frequencies follow real-world patterns.

In [ ]:
# Define target distribution based on ZIP code digit frequencies
# This simulates real-world deployment where certain digits are more common
print("🌍 PRODUCTION DEPLOYMENT SCENARIO: ZIP CODE PROCESSING")
print("="*65)

# Realistic ZIP code digit distribution (approximate US patterns)
zip_code_distribution = np.array([
    0.15,   # 0: Common in many ZIP codes
    0.12,   # 1: Frequent 
    0.11,   # 2: Frequent
    0.09,   # 3: Moderate
    0.09,   # 4: Moderate  
    0.08,   # 5: Moderate
    0.10,   # 6: Moderate
    0.08,   # 7: Less common
    0.09,   # 8: Moderate
    0.09    # 9: Moderate
])

print("Target distribution for ZIP code processing:")
for digit, freq in enumerate(zip_code_distribution):
    print(f"  Digit {digit}: {freq:.3f} ({freq*100:.1f}%)")

print(f"\nDistribution shift from training:")
distribution_shift = zip_code_distribution - training_distribution
for digit, shift in enumerate(distribution_shift):
    direction = "↑" if shift > 0 else "↓" if shift < 0 else "→"
    print(f"  Digit {digit}: {shift:+.3f} {direction}")

# Business impact analysis
print(f"\n💼 BUSINESS IMPACT ANALYSIS:")
print(f"   • Mail routing accuracy critical for delivery performance")
print(f"   • Misclassified digits lead to delivery delays and customer complaints")
print(f"   • Need probability estimates aligned with actual ZIP code patterns")
print(f"   • Regulatory requirements for postal service reliability")

# Target marginals for calibration
n_test_samples = len(y_test)
target_marginals = zip_code_distribution * n_test_samples

print(f"\nCalibration targets:")
print(f"  Total samples: {n_test_samples}")
print(f"  Target marginals: {target_marginals}")

## Rank-Preserving Calibration

In [ ]:
# Apply rank-preserving calibration
print("🔧 APPLYING RANK-PRESERVING CALIBRATION")
print("="*50)

# Calibrate probabilities
result = calibrate_dykstra(
    P=y_proba, 
    M=target_marginals,
    max_iters=2000,
    tol=1e-7,
    verbose=True
)

y_proba_calibrated = result.Q
print(f"\nCalibration completed successfully!")
print(f"  Converged: {result.converged}")
print(f"  Iterations: {result.iterations}")
print(f"  Final objective: {result.objective:.2e}")

# Verify calibration worked
calibrated_marginals = np.sum(y_proba_calibrated, axis=0)
print(f"\n✅ CALIBRATION VERIFICATION:")
print(f"Target vs Achieved marginals:")
for digit in range(10):
    target = target_marginals[digit]
    achieved = calibrated_marginals[digit]
    error = abs(achieved - target)
    print(f"  Digit {digit}: {target:.1f} → {achieved:.1f} (error: {error:.2e})")

max_marginal_error = np.max(np.abs(calibrated_marginals - target_marginals))
print(f"\nMaximum marginal constraint violation: {max_marginal_error:.2e}")

## Impact Analysis and Visualization

In [ ]:
# Comprehensive analysis of calibration impact
print("📈 CALIBRATION IMPACT ANALYSIS")
print("="*40)

# 1. Ranking preservation
from scipy.stats import spearmanr

# Check if rankings are preserved for each sample
spearman_correlations = []
for i in range(len(y_test)):
    corr, _ = spearmanr(y_proba[i], y_proba_calibrated[i])
    spearman_correlations.append(corr)

spearman_correlations = np.array(spearman_correlations)
perfect_rank_preservation = np.sum(np.isclose(spearman_correlations, 1.0, atol=1e-10))

print(f"RANK PRESERVATION ANALYSIS:")
print(f"  Perfect rank preservation: {perfect_rank_preservation}/{len(y_test)} samples")
print(f"  Mean Spearman correlation: {np.mean(spearman_correlations):.6f}")
print(f"  Min Spearman correlation: {np.min(spearman_correlations):.6f}")
print(f"  Samples with correlation < 0.999: {np.sum(spearman_correlations < 0.999)}")

# 2. Prediction changes
original_predictions = np.argmax(y_proba, axis=1)
calibrated_predictions = np.argmax(y_proba_calibrated, axis=1)
prediction_changes = np.sum(original_predictions != calibrated_predictions)

print(f"\nPREDICTION IMPACT:")
print(f"  Total prediction changes: {prediction_changes}/{len(y_test)}")
print(f"  Prediction stability: {(1 - prediction_changes/len(y_test))*100:.1f}%")

if prediction_changes > 0:
    changed_indices = np.where(original_predictions != calibrated_predictions)[0]
    print(f"  Changed predictions involve digits:")
    for idx in changed_indices[:5]:  # Show first 5 changes
        orig = original_predictions[idx]
        calib = calibrated_predictions[idx]
        true_label = y_test[idx]
        print(f"    Sample {idx}: {orig} → {calib} (true: {true_label})")

# 3. Accuracy comparison
original_accuracy = accuracy_score(y_test, original_predictions)
calibrated_accuracy = accuracy_score(y_test, calibrated_predictions)

print(f"\nACCURACY COMPARISON:")
print(f"  Original accuracy: {original_accuracy:.4f}")
print(f"  Calibrated accuracy: {calibrated_accuracy:.4f}")
print(f"  Accuracy change: {calibrated_accuracy - original_accuracy:+.4f}")

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(3, 3, figsize=(18, 15))
colors = plt.cm.tab10(np.linspace(0, 1, 10))

# 1. Marginal comparison
x_pos = np.arange(10)
width = 0.25

axes[0, 0].bar(x_pos - width, training_distribution, width, 
               label='Training', alpha=0.8, color='skyblue')
axes[0, 0].bar(x_pos, current_marginals, width, 
               label='Original Model', alpha=0.8, color='orange')
axes[0, 0].bar(x_pos + width, zip_code_distribution, width, 
               label='Target (ZIP codes)', alpha=0.8, color='green')
axes[0, 0].set_xlabel('Digit')
axes[0, 0].set_ylabel('Probability Mass')
axes[0, 0].set_title('Distribution Comparison')
axes[0, 0].legend()
axes[0, 0].set_xticks(x_pos)

# 2. Calibration accuracy per digit
achieved_distribution = calibrated_marginals / n_test_samples
calibration_errors = np.abs(achieved_distribution - zip_code_distribution)

bars = axes[0, 1].bar(x_pos, calibration_errors, color=colors, alpha=0.7)
axes[0, 1].set_xlabel('Digit')
axes[0, 1].set_ylabel('Absolute Error')
axes[0, 1].set_title('Calibration Accuracy by Digit')
axes[0, 1].set_xticks(x_pos)
axes[0, 1].set_yscale('log')

# 3. Probability change distribution
prob_changes = y_proba_calibrated - y_proba
axes[0, 2].hist(prob_changes.flatten(), bins=50, alpha=0.7, 
                density=True, color='purple')
axes[0, 2].axvline(0, color='black', linestyle='--')
axes[0, 2].set_xlabel('Probability Change')
axes[0, 2].set_ylabel('Density')
axes[0, 2].set_title('Distribution of Probability Changes')

# 4. Reliability diagram (calibration curve)
def plot_reliability_diagram(y_true, y_proba, ax, title):
    n_classes = y_proba.shape[1]
    for digit in range(n_classes):
        y_binary = (y_true == digit).astype(int)
        if np.sum(y_binary) > 0:  # Only plot if class exists
            fraction_of_positives, mean_predicted_value = calibration_curve(
                y_binary, y_proba[:, digit], n_bins=10
            )
            ax.plot(mean_predicted_value, fraction_of_positives, 
                   's-', label=f'Digit {digit}', color=colors[digit], alpha=0.7)
    
    ax.plot([0, 1], [0, 1], 'k:', label='Perfect calibration')
    ax.set_xlabel('Mean Predicted Probability')
    ax.set_ylabel('Fraction of Positives')
    ax.set_title(title)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plot_reliability_diagram(y_test, y_proba, axes[1, 0], 'Original Model Calibration')
plot_reliability_diagram(y_test, y_proba_calibrated, axes[1, 1], 'Calibrated Model')

# 5. Per-class probability changes
prob_changes = y_proba_calibrated - y_proba
for digit in range(10):
    axes[1, 2].hist(prob_changes[:, digit], bins=20, alpha=0.7, 
                   label=f'Digit {digit}', color=colors[digit], density=True)

axes[1, 2].axvline(0, color='black', linestyle='--')
axes[1, 2].set_xlabel('Probability Change')
axes[1, 2].set_ylabel('Density')
axes[1, 2].set_title('Distribution of Changes by Digit')
axes[1, 2].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# 6. Confusion matrix comparison
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm_original = confusion_matrix(y_test, original_predictions)
cm_calibrated = confusion_matrix(y_test, calibrated_predictions)

# Normalize for better comparison
cm_original_norm = cm_original.astype('float') / cm_original.sum(axis=1)[:, np.newaxis]
cm_calibrated_norm = cm_calibrated.astype('float') / cm_calibrated.sum(axis=1)[:, np.newaxis]

im1 = axes[2, 0].imshow(cm_original_norm, interpolation='nearest', cmap=plt.cm.Blues)
axes[2, 0].set_title('Original Predictions')
axes[2, 0].set_xlabel('Predicted Digit')
axes[2, 0].set_ylabel('True Digit')
axes[2, 0].set_xticks(range(10))
axes[2, 0].set_yticks(range(10))

im2 = axes[2, 1].imshow(cm_calibrated_norm, interpolation='nearest', cmap=plt.cm.Blues)
axes[2, 1].set_title('Calibrated Predictions')
axes[2, 1].set_xlabel('Predicted Digit')
axes[2, 1].set_ylabel('True Digit')
axes[2, 1].set_xticks(range(10))
axes[2, 1].set_yticks(range(10))

# 7. Difference in confusion matrices
cm_diff = cm_calibrated_norm - cm_original_norm
im3 = axes[2, 2].imshow(cm_diff, interpolation='nearest', cmap=plt.cm.RdBu, 
                        vmin=-np.max(np.abs(cm_diff)), vmax=np.max(np.abs(cm_diff)))
axes[2, 2].set_title('Difference (Calibrated - Original)')
axes[2, 2].set_xlabel('Predicted Digit')
axes[2, 2].set_ylabel('True Digit')
axes[2, 2].set_xticks(range(10))
axes[2, 2].set_yticks(range(10))

plt.tight_layout()
plt.show()

## Business Impact Assessment

In [ ]:
# Business impact analysis for computer vision deployment
print("💰 BUSINESS IMPACT ASSESSMENT")
print("="*50)

# Simulate business metrics
n_daily_images = 50000  # Images processed per day
n_annual_images = n_daily_images * 365

# Cost parameters
cost_per_misclassification = 2.50  # Cost of routing error
cost_per_manual_review = 0.15     # Human verification cost
revenue_per_correct_classification = 0.05  # Processing fee

# Calculate error rates and associated costs
original_error_rate = 1 - original_accuracy
calibrated_error_rate = 1 - calibrated_accuracy

print(f"📊 OPERATIONAL METRICS:")
print(f"   • Daily image volume: {n_daily_images:,}")
print(f"   • Annual image volume: {n_annual_images:,}")
print(f"   • Original error rate: {original_error_rate:.4f} ({original_error_rate*100:.2f}%)")
print(f"   • Calibrated error rate: {calibrated_error_rate:.4f} ({calibrated_error_rate*100:.2f}%)")

# Annual cost comparison
original_annual_errors = n_annual_images * original_error_rate
calibrated_annual_errors = n_annual_images * calibrated_error_rate
error_reduction = original_annual_errors - calibrated_annual_errors

original_error_cost = original_annual_errors * cost_per_misclassification
calibrated_error_cost = calibrated_annual_errors * cost_per_misclassification
annual_cost_savings = original_error_cost - calibrated_error_cost

print(f"\n💵 ANNUAL FINANCIAL IMPACT:")
print(f"   • Original annual errors: {original_annual_errors:,.0f}")
print(f"   • Calibrated annual errors: {calibrated_annual_errors:,.0f}")
print(f"   • Error reduction: {error_reduction:,.0f} ({(error_reduction/original_annual_errors)*100:.1f}%)")
print(f"   • Annual cost savings: ${annual_cost_savings:,.2f}")

# Confidence-based routing analysis
# Use calibrated probabilities to determine which images need manual review
confidence_threshold = 0.95
max_calibrated_probs = np.max(y_proba_calibrated, axis=1)
max_original_probs = np.max(y_proba, axis=1)

high_conf_original = np.sum(max_original_probs >= confidence_threshold)
high_conf_calibrated = np.sum(max_calibrated_probs >= confidence_threshold)

manual_review_original = len(y_test) - high_conf_original
manual_review_calibrated = len(y_test) - high_conf_calibrated

print(f"\n🔍 CONFIDENCE-BASED ROUTING (threshold = {confidence_threshold}):")
print(f"   • Original: {high_conf_original}/{len(y_test)} auto-processed")
print(f"   • Calibrated: {high_conf_calibrated}/{len(y_test)} auto-processed")
print(f"   • Manual review reduction: {manual_review_original - manual_review_calibrated} samples")

# Scale to annual volume
annual_manual_original = (manual_review_original / len(y_test)) * n_annual_images
annual_manual_calibrated = (manual_review_calibrated / len(y_test)) * n_annual_images
annual_manual_savings = annual_manual_original - annual_manual_calibrated

manual_cost_savings = annual_manual_savings * cost_per_manual_review

print(f"   • Annual manual review cost savings: ${manual_cost_savings:,.2f}")

# Total business impact
total_annual_savings = annual_cost_savings + manual_cost_savings
roi_percentage = (total_annual_savings / (n_annual_images * revenue_per_correct_classification)) * 100

print(f"\n🎯 TOTAL BUSINESS IMPACT:")
print(f"   • Total annual savings: ${total_annual_savings:,.2f}")
print(f"   • ROI on processing volume: {roi_percentage:.2f}%")
print(f"   • Payback period: Immediate (operational efficiency)")

# Deployment recommendations
print(f"\n🚀 DEPLOYMENT RECOMMENDATIONS:")
deployment_recommendations = [
    "Implement calibrated model in production OCR pipeline",
    "Use confidence thresholds for automated vs manual routing",
    "Monitor real-world digit distribution shifts over time",
    "Establish periodic recalibration based on seasonal patterns",
    "Extend calibration to other postal code formats (international)",
    "Consider ensemble methods for further accuracy improvements",
    "Implement A/B testing framework for continuous optimization"
]

for i, rec in enumerate(deployment_recommendations, 1):
    print(f"   {i}. {rec}")

# Risk assessment
print(f"\n⚠️  IMPLEMENTATION CONSIDERATIONS:")
considerations = [
    "Validate calibration on recent ZIP code data before deployment",
    "Monitor for concept drift in digit distribution patterns",
    "Ensure compliance with postal service accuracy requirements",
    "Plan for emergency fallback to original model if needed",
    "Train operations team on new confidence-based routing logic"
]

for consideration in considerations:
    print(f"   • {consideration}")

print(f"\n📈 PERFORMANCE GUARANTEES:")
print(f"   • Rank preservation: Perfect ordering maintained (correlation = {np.mean(spearman_correlations):.6f})")
print(f"   • Constraint satisfaction: Maximum marginal error < {np.max(np.abs(calibrated_marginals - target_marginals)):.2e}")
print(f"   • Convergence: Algorithm converged in {result.iterations} iterations")
print(f"   • Stability: {(1 - prediction_changes/n_test_samples)*100:.1f}% of predictions unchanged")

# Calibration quality check
final_row_errors = np.abs(y_proba_calibrated.sum(axis=1) - 1.0)
final_col_errors = np.abs(calibrated_marginals - target_marginals)

print(f"\n✅ QUALITY ASSURANCE:")
print(f"   • Maximum row constraint violation: {np.max(final_row_errors):.2e}")
print(f"   • Maximum column constraint violation: {np.max(final_col_errors):.2e}")
print(f"   • Calibration converged: {'Yes' if result.converged else 'No'}")
print(f"   • All mathematical constraints satisfied within numerical precision")

print(f"\n🚀 NEXT STEPS:")
print(f"   1. Deploy calibrated model in staging environment")
print(f"   2. Monitor real-world performance against baseline")
print(f"   3. Collect feedback on decision quality improvements")
print(f"   4. Establish periodic recalibration protocols")
print(f"   5. Scale to additional computer vision applications")

## Next Steps

This example demonstrated rank-preserving calibration for computer vision applications. The same principles apply broadly to:

- **Medical imaging** with population-specific disease prevalence
- **Autonomous systems** requiring calibrated uncertainty for safety
- **Industrial automation** with domain-specific defect rates
- **Content moderation** across platforms with different content distributions
- **Retail applications** with store or region-specific product mix

The key insight is that rank-preserving calibration maintains the model's core discriminative ability while adapting the probability estimates to match deployment conditions.

For more examples in different domains, see the other notebooks:
- Medical diagnosis with clinical population shifts
- Text classification with domain adaptation
- Financial risk assessment with portfolio-specific distributions
- Survey reweighting for demographic correction